# Exploratory Spatial Data Analysis of Disaster-Tweets

With this notebook, you can analyse tweets that were posted in the same area and on the same day as the Napa earthquake in 2014 (see https://en.wikipedia.org/wiki/2014_South_Napa_earthquake for details).  

### Assignment

Create a new markdown cell to answer the following questions:

* How many tweets are in the dataset?
* What words are the most frequent words in the wordcloud (related to the size of the words)?
* Perform topic modelling with and without preprocessing. What could you observe?
* Which topics could you identify in the datasets? Can you label some of the topics?
* What is the min and max date of the dataset?
* How does the time-series of the disaster-related topic and the overall dataset differ?
* How does the heatmap of the disaster-related topic and the overall dataset differ?

When you have answered the questions, download your notebook as HTML-file and check if it worked.
File -> Download as -> HTML

Hint: You do not have to program something by yourself, but you have to understand the code (change some variables) and look for the answers to the questions.


## Load dataset and libraries

In [ ]:
%matplotlib inline

In [ ]:
from preprocessing import *
import gensim
from nltk.stem.porter import *
import nltk
from nltk.corpus import stopwords
from gensim import corpora
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
import folium
import pyLDAvis.gensim
import statistics
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
df = pd.read_csv('../tweets/napa_tweets.csv', sep=',', error_bad_lines=False, index_col=False, warn_bad_lines=False)
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df.head(5)

In [ ]:
df.shape

## Natural language processing

In [ ]:
wordcloud = WordCloud(background_color='white').generate(' '.join(df['tweet_text']))
plt.figure(figsize=(20, 20))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
?WordCloud

Download nltk Ressources.

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

### Transform tweets text into usable format

In [ ]:
#tweets' text as list
tweets_text = df['tweet_text'].tolist()
#lowercase
tweets_text=[tweet.lower() for tweet in tweets_text]

#remove URLs
remove_url_regex = r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b'
tweets_text = filter_tweets_before_tokenization(tweets_text, remove_url_regex)

#tokenization
tweets_text=[nltk.word_tokenize(tweet) for tweet in tweets_text]

### Text preprocessing

Test multiple preprocessing procedures and observe their impact on the analysis results

In [ ]:
#remove special characters
remove_sc_regex = r'[^A-Za-z ]+'
tweets_text = filter_tweets_after_tokenization(tweets_text, remove_sc_regex)

# remove short words
remove_short_words_regex = r'\W*\b\w{1,3}\b'
tweets_text = filter_tweets_after_tokenization(tweets_text, remove_short_words_regex)

# Remove all user names in the tweet text
user_names_regex = r"@\S+"
tweets_text = filter_tweets_after_tokenization(tweets_text,user_names_regex)

#increase keyword frequency by aggregating similar keywords
# check the order if preprocessing routine! e.g. stemming would effect the performance of synonym handling
#disaster = 'hurrican'
#disaster_terms = ['hurricane', 'hurricaneharvey', 'hurricane_harvey', 'flood', 'storm']
#tweets_text = synonym_handling(tweets_text, disaster, disaster_terms)

#Remove unique words that appear only once in the dataset
frequency = getFrequency(tweets_text)
min_frequency_words = 2
tweets_text = [[token for token in tweet if frequency[token] > min_frequency_words] for tweet in tweets_text]

# Remove stop words
# You need to download the stopwords
from nltk.corpus import PlaintextCorpusReader
stoplist = set(stopwords.words('english'))
tweets_text = [[word for word in document if word not in stoplist] for document in tweets_text]

#Stemming
stemmer = PorterStemmer()
#stemmer = SnowballStemmer("english")
tweets_text = [[stemmer.stem(word) for word in sub_list] for sub_list in tweets_text]

#remove empty strings
tweets_text = [[word for word in document if word] for document in tweets_text]

tweets_text[:10]

Create corpus and dictionary for LDA

In [ ]:
dict = gensim.corpora.Dictionary(tweets_text)
corpus = [dict.doc2bow(text) for text in tweets_text]

Train Model

In [ ]:
num_topics= 10
alpha = 0.0001
eta= 0.0001
passes = 10
lda = gensim.models.LdaMulticore(corpus, id2word=dict, num_topics= num_topics, alpha = alpha, eta= eta, passes = passes)

Show top words of topics

In [ ]:
top_words = 5

#show top words of topics
for t in range(lda.num_topics):
    print('topic {}: '.format(t+1) + ', '.join([v[0] for v in lda.show_topic(t, top_words)]))

#show top words of topics with probabilities  
#for t in range(lda.num_topics):
#   print('topic {}: '.format(t+1) + ', '.join([v[0] + " (" + str(v[1]) + ")" for v in lda.show_topic(t, top_words)]))


### Visualise topics and check relation between them
If the window is not big enough, you can enlarge it with Cell -> Current Outputs -> Toggle Scrolling 

In [ ]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(lda, corpus, dict, sort_topics=False)
pyLDAvis.display(vis)

Identify disaster-related topic and classify tweets accordingly

In [ ]:
document_topic_list = list(lda.get_document_topics(corpus))
classified_tweets =[max(document, key=lambda x: x[1]) for document in document_topic_list]
topics = [top_prob[0]+1 for top_prob in classified_tweets]
probabilites = [top_prob[1] for top_prob in classified_tweets]
df['topics'] = topics
df['probabilities'] = probabilites

Check classified tweets

In [ ]:
topic_number = 6
df.loc[df['topics'] == topic_number].head(10)

## Check time-series

In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
print(min(df['time']))
print(max(df['time']))

In [ ]:
topic_number = 8
figure, axes = plt.subplots(1, 2,figsize=(20,8))

df_sum = df['time'].value_counts().resample('T').sum()
ax = df_sum.plot(label='Number of Tweets',ax=axes[0])
axes[0].grid()
axes[0].set_ylabel("#Tweets")

df_topic = df.loc[df['topics'] == topic_number]
df_sum = df_topic['time'].value_counts().resample('T').sum()
ax = df_sum.plot(label='Number of Tweets',ax=axes[1])
axes[1].grid()
axes[1].set_ylabel("#Tweets")

## Check geospatial distribution

In [ ]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
y = statistics.mean(df['latitude']) 
x = statistics.mean(df['longitude']) 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

df['count'] = 1
base_map = generateBaseMap([y,x],8)
HeatMap(data=df[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
display(base_map)

In [ ]:
topic_numbers = [4,8]
base_maps = []
for topic_number in topic_numbers:
    df_topic = df.loc[df['topics'] == topic_number]
    base_map = generateBaseMap([y,x],8)
    HeatMap(data=df_topic[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
    base_maps.append(base_map)

htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           .format(base_maps[0].get_root().render().replace('"', '&quot;'),500,500,
                   base_maps[1].get_root().render().replace('"', '&quot;'),500,500))
display(htmlmap)

More information

* [Latent Dirichlet Allocation](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)
* [Combining machine-learning topic models and spatiotemporal analysis of social media data for disaster footprint and damage assessment](https://www.tandfonline.com/doi/full/10.1080/15230406.2017.1356242)
* [Gensim](https://radimrehurek.com/gensim/)